In [14]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("DataExplore") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory", "2g") \
.getOrCreate()

Veri Seti OkumA Oncesi Kontroller

In [15]:
adult_train_df= spark.read \
.option("header","True") \
.option("inferSchema", "True") \
.option("sep",",") \
.csv("adult.data") \

In [16]:
adult_test_df= spark.read \
.option("header","True") \
.option("inferSchema", "True") \
.option("sep",",") \
.csv("adult.test") \

In [17]:
adult_train_df.show(5)

+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass|  fnlwgt| education|education_num|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|output|
+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516.0| Bachelors|         13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
| 50| Self-emp-not-inc| 83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|         0.0|         0.0|          13.0| United-States| <=50K|
| 38|          Private|215646.0|   HS-grad|       

In [20]:
adult_train_df.limit(5).toPandas().head() #veriyi net goremıyorsak bu pandas data frame cevırıp gorebılırız


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [21]:
adult_all_df=adult_train_df.union(adult_test_df)
adult_all_df.limit(5).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [22]:
adult_all_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- output: string (nullable = true)



NUMERIK DEGISKENLERIN ISTATİSTİKLERİ

In [23]:
adult_all_df.describe(["age","fnlwgt","education_num","capital_gain","capital_loss","hours_per_week","hours_per_week"])\
.toPandas().T.head()

,0,1,2,3,4
summary,count,mean,stddev,min,max
age,48842,38.64358543876172,13.710509934443563,17,90
fnlwgt,48842,189664.13459727284,105604.02542315733,12285.0,1490400.0
education_num,48842,10.078088530363212,2.570972755592263,1.0,16.0
capital_gain,48842,1079.0676262233324,7452.019057655401,0.0,99999.0


kATEGORİK DEGİSKENLERİ İNCELEME


In [27]:
from pyspark.sql import functions as F

In [33]:
adult_all_df.groupBy(F.col("workclass")) \
.agg({"*":"count"}) \
.toPandas().head(10)

,workclass,count(1)
0,State-gov,1981
1,Federal-gov,1432
2,Self-emp-not-inc,3862
3,Local-gov,3136
4,Private,33906
5,?,2799
6,Self-emp-inc,1695
7,Without-pay,21
8,Never-worked,10


In [35]:
adult_all_df.groupBy(F.col("education")).agg({"*":"count"}).toPandas().head(20)

,education,count(1)
0,Prof-school,834
1,10th,1389
2,7th-8th,955
3,5th-6th,509
4,Assoc-acdm,1601
5,Assoc-voc,2061
6,Masters,2657
7,12th,657
8,Preschool,83
9,9th,756
